In [1]:
pip -q install -U autoawq transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00


In [18]:
# import torch
# from awq import AutoAWQForCausalLM
# from transformers import AutoTokenizer, AutoConfig
# import gc
# import os

# # Configure environment
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# print("Checking PyTorch and CUDA versions...")
# print(f"PyTorch version: {torch.__version__}")
# print(f"CUDA available: {torch.cuda.is_available()}")
# if torch.cuda.is_available():
#     print(f"CUDA version: {torch.version.cuda}")
#     print(f"GPU: {torch.cuda.get_device_name()}")

# # Clear any existing cache
# if torch.cuda.is_available():
#     torch.cuda.empty_cache()
# gc.collect()

# print("Loading tokenizer...")
# tok = AutoTokenizer.from_pretrained(
#     base_model,
#     use_fast=True,
#     trust_remote_code=True
# )
# if tok.pad_token is None:
#     tok.pad_token = tok.eos_token

# print("Loading model...")
# # Load model with specific configurations to avoid attention issues
# mdl = AutoAWQForCausalLM.from_pretrained(
#     base_model,
#     low_cpu_mem_usage=True,
#     use_cache=False,
#     trust_remote_code=True,
#     torch_dtype=torch.float16,  # Use fp16 explicitly
#     device_map={"": 0} if torch.cuda.is_available() else "cpu"  # More specific device mapping
# )

# # Set model to eval mode
# mdl.eval()

# # AWQ quantization config
# quant_config = {
#     "zero_point": True,
#     "q_group_size": 128,
#     "w_bit": 4,
#     "version": "GEMM"
# }

# # Create more diverse and shorter calibration data
# print("Preparing calibration data...")
# calib_texts = [
#     "The quick brown fox jumps over the lazy dog.",
#     "Machine learning models process data efficiently.",
#     "Natural language understanding is advancing rapidly.",
#     "Deep neural networks learn complex patterns.",
#     "Artificial intelligence transforms technology.",
#     "Computer vision recognizes objects accurately.",
#     "Robotics integrates sensors and actuators.",
#     "Algorithm optimization improves performance significantly.",
#     "Data science extracts meaningful insights.",
#     "Software engineering creates reliable systems."
# ] * 10  # 100 samples total

# print("Starting quantization...")
# try:
#     # Use minimal configuration to avoid batch size issues
#     mdl.quantize(
#         tok,
#         quant_config=quant_config,
#         calib_data=calib_texts,
#         max_calib_seq_len=128,     # Reduced sequence length
#         max_calib_samples=50,      # Reduced sample count
#         n_parallel_calib_samples=1 # Keep sequential processing
#     )

#     print("Quantization completed successfully!")

#     # Save the quantized model
#     out_dir = "tinyllama-1.1b-awq"
#     print(f"Saving model to {out_dir}...")

#     mdl.save_quantized(out_dir, safetensors=True)
#     tok.save_pretrained(out_dir)

#     # Save config
#     config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
#     config.save_pretrained(out_dir)

#     print(f"Model successfully quantized and saved to {out_dir}")

# except Exception as e:
#     print(f"Quantization failed with error: {str(e)}")
#     print("\nTrying alternative approach with different model loading...")

#     # Alternative approach: Load model differently
#     del mdl
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#     gc.collect()

#     try:
#         # Try loading without device_map first
#         mdl = AutoAWQForCausalLM.from_pretrained(
#             base_model,
#             low_cpu_mem_usage=True,
#             trust_remote_code=True,
#             use_cache=False
#         )

#         # Move to device manually if CUDA is available
#         if torch.cuda.is_available():
#             mdl = mdl.cuda()

#         mdl.eval()

#         # Try with even smaller calibration parameters
#         print("Attempting quantization with minimal parameters...")
#         mdl.quantize(
#             tok,
#             quant_config=quant_config,
#             calib_data=calib_texts[:20],  # Only use first 20 samples
#             max_calib_seq_len=64,         # Even smaller sequence length
#             max_calib_samples=20,         # Minimal samples
#             n_parallel_calib_samples=1
#         )

#         out_dir = "tinyllama-1.1b-awq"
#         mdl.save_quantized(out_dir, safetensors=True)
#         tok.save_pretrained(out_dir)

#         config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
#         config.save_pretrained(out_dir)

#         print(f"Model successfully quantized with alternative approach and saved to {out_dir}")

#     except Exception as e2:
#         print(f"Alternative approach also failed: {str(e2)}")
#         print("\nConsider using a pre-quantized model instead:")
#         print("TheBloke/TinyLlama-1.1B-Chat-v1.0-AWQ")

# finally:
#     # Cleanup
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#     gc.collect()

Checking PyTorch and CUDA versions...
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU: Tesla T4
Loading tokenizer...
Loading model...


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Preparing calibration data...
Starting quantization...


AWQ:   0%|          | 0/22 [00:00<?, ?it/s]


Quantization failed with error: output with shape [1, 32, 128, 128] doesn't match the broadcast shape [3, 32, 128, 128]

Trying alternative approach with different model loading...


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Attempting quantization with minimal parameters...


AWQ:   0%|          | 0/22 [00:00<?, ?it/s]

Alternative approach also failed: output with shape [1, 32, 64, 64] doesn't match the broadcast shape [2, 32, 64, 64]

Consider using a pre-quantized model instead:
TheBloke/TinyLlama-1.1B-Chat-v1.0-AWQ


In [3]:
import torch
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer, AutoConfig
import gc
import os

# Configure environment
os.environ["TOKENIZERS_PARALLELISM"] = "false"

base_model = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

print("Checking PyTorch and CUDA versions...")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name()}")

# Clear any existing cache
if torch.cuda.is_available():
    torch.cuda.empty_cache()
gc.collect()

print("Loading tokenizer...")
tok = AutoTokenizer.from_pretrained(
    base_model,
    use_fast=True,
    trust_remote_code=True
)
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

print("Loading model...")
# Load model with specific configurations to avoid attention issues
mdl = AutoAWQForCausalLM.from_pretrained(
    base_model,
    low_cpu_mem_usage=True,
    use_cache=False,
    trust_remote_code=True,
    torch_dtype=torch.float16,  # Use fp16 explicitly
    device_map={"": 0} if torch.cuda.is_available() else "cpu"  # More specific device mapping
)

# Set model to eval mode
mdl.eval()

# AWQ quantization config
quant_config = {
    "zero_point": True,
    "q_group_size": 128,
    "w_bit": 4,
    "version": "GEMM"
}

# Create more diverse and shorter calibration data
print("Preparing calibration data...")
calib_texts = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning models process data efficiently.",
    "Natural language understanding is advancing rapidly.",
    "Deep neural networks learn complex patterns.",
    "Artificial intelligence transforms technology.",
    "Computer vision recognizes objects accurately.",
    "Robotics integrates sensors and actuators.",
    "Algorithm optimization improves performance significantly.",
    "Data science extracts meaningful insights.",
    "Software engineering creates reliable systems."
] * 10  # 100 samples total

print("Starting quantization...")
try:
    # Use minimal configuration to avoid batch size issues
    mdl.quantize(
        tok,
        quant_config=quant_config,
        calib_data=calib_texts,
        max_calib_seq_len=128,     # Reduced sequence length
        max_calib_samples=50,      # Reduced sample count
        n_parallel_calib_samples=1 # Keep sequential processing
    )

    print("Quantization completed successfully!")

    # Save the quantized model
    out_dir = "tinyllama-1.1b-awq"
    print(f"Saving model to {out_dir}...")

    mdl.save_quantized(out_dir, safetensors=True)
    tok.save_pretrained(out_dir)

    # Save config
    config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
    config.save_pretrained(out_dir)

    print(f"Model successfully quantized and saved to {out_dir}")

except Exception as e:
    print(f"Quantization failed with error: {str(e)}")
    print("\nTrying alternative approach with different model loading...")

    # Alternative approach: Load model differently
    del mdl
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

    try:
        # Try loading without device_map first
        mdl = AutoAWQForCausalLM.from_pretrained(
            base_model,
            low_cpu_mem_usage=True,
            trust_remote_code=True,
            use_cache=False
        )

        # Move to device manually if CUDA is available
        if torch.cuda.is_available():
            mdl = mdl.cuda()

        mdl.eval()

        # Try with even smaller calibration parameters
        print("Attempting quantization with minimal parameters...")
        mdl.quantize(
            tok,
            quant_config=quant_config,
            calib_data=calib_texts[:20],  # Only use first 20 samples
            max_calib_seq_len=64,         # Even smaller sequence length
            max_calib_samples=20,         # Minimal samples
            n_parallel_calib_samples=1
        )

        out_dir = "tinyllama-1.1b-awq"
        mdl.save_quantized(out_dir, safetensors=True)
        tok.save_pretrained(out_dir)

        config = AutoConfig.from_pretrained(base_model, trust_remote_code=True)
        config.save_pretrained(out_dir)

        print(f"Model successfully quantized with alternative approach and saved to {out_dir}")

    except Exception as e2:
        print(f"Alternative approach also failed: {str(e2)}")
        print("\nConsider using a pre-quantized model instead:")
        print("TheBloke/TinyLlama-1.1B-Chat-v1.0-AWQ")

finally:
    # Cleanup
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

/usr/local/lib/python3.11/dist-packages/awq/__init__.py:21: DeprecationWarning: 
I have left this message as the final dev message to help you transition.

Important Notice:
- AutoAWQ is officially deprecated and will no longer be maintained.
- The last tested configuration used Torch 2.6.0 and Transformers 4.51.3.
- If future versions of Transformers break AutoAWQ compatibility, please report the issue to the Transformers project.

Alternative:
- AutoAWQ has been adopted by the vLLM Project: https://github.com/vllm-project/llm-compressor

For further inquiries, feel free to reach out:
- X: https://x.com/casper_hansen_
- LinkedIn: https://www.linkedin.com/in/casper-hansen-804005170/

  warnings.warn(_FINAL_DEV_MESSAGE, category=DeprecationWarning, stacklevel=1)


Checking PyTorch and CUDA versions...
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU: Tesla T4
Loading tokenizer...


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Quantization failed with error: The expanded size of the tensor (1) must match the existing size (3) at non-singleton dimension 0.  Target sizes: [1, 32, 128, 128].  Tensor sizes: [3, 1, 128, 128]

Trying alternative approach with different model loading...


Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

Attempting quantization with minimal parameters...


AWQ:   0%|          | 0/22 [00:00<?, ?it/s]


Alternative approach also failed: output with shape [1, 32, 64, 64] doesn't match the broadcast shape [2, 32, 64, 64]

Consider using a pre-quantized model instead:
TheBloke/TinyLlama-1.1B-Chat-v1.0-AWQ
